In [1]:
import requests 
import pandas as pd
import numpy as np
import json
import warnings
warnings.simplefilter("ignore")

In [5]:
client_id= 'cab818deef4a4d9c8024b58ae0c6baf7'
client_secret= '7eaf3d88c03e4a78a1e9a33bbb6fc871'
access_token= auth_spotify(client_id, client_secret)

In [4]:
# get authentification


def auth_spotify (client_id, client_secret):
    URl= 'https://accounts.spotify.com/api/token'

    auth_response = requests.post(URl, {
        'grant_type': 'client_credentials',
        'client_id': client_id,
        'client_secret': client_secret,

    })
    
    auth_response= auth_response.json()
    access_token= auth_response['access_token']
    
    return access_token
    
access_token= auth_spotify(client_id, client_secret)

In [6]:
# get playlist item

def get_playlist_tracks (access_token, playlist_id):
    header= {'Authorization': "Bearer {}".format(access_token)}
    URl_base = 'https://api.spotify.com/v1/'
    
    playlist = requests.get(URl_base + 'playlists/' + playlist_id + '/tracks', headers=header)
    playlist = playlist.json()
    return playlist

In [7]:
# get track's audio features

def get_track_feature(track_id):
    header= {'Authorization': "Bearer {}".format(access_token)}
    URl_base = 'https://api.spotify.com/v1/'
 
    
    track_feature = requests.get(URl_base + 'audio-features/' + track_id, headers=header)
    track_feature = track_feature.json()
    return track_feature

In [8]:
# get track

def get_unique_track (track_id):
    header= {'Authorization': "Bearer {}".format(access_token)}
    URl_base = 'https://api.spotify.com/v1/'
    
    track = requests.get(URl_base + 'tracks/' + track_id, headers=header)
    track = track.json()
    return track

In [9]:
# get artist

def get_unique_artist(artist_id):
    header= {'Authorization': "Bearer {}".format(access_token)}
    URl_base = 'https://api.spotify.com/v1/'
 
    
    artist_feature = requests.get(URl_base + 'artists/' + artist_id, headers=header)
    artist_feature = artist_feature.json()
    return artist_feature

In [10]:
# get a song likes me or not

def get_target_track (tracks_id):
    header= {'Authorization': "Bearer {}".format(authorization_scope)}
    
    contain= "contains?ids={}".format(tracks_id)
    URl_base = 'https://api.spotify.com/v1/me/'
    
    track = requests.get(URl_base + 'tracks/' + contain, headers=header)
    track= track.json()
    
    if track[0]== True:
        track= 1
    else:
        track= 0
    return track

In [11]:
def add_playlist(playlist, df_playlist=None):
    # Initialize a list to hold the rows of data
    rows = []
    
    for item in playlist['tracks']['items']:
        track_id = item['track']['id']
        track_name = item['track']['name']
        
        track = get_unique_track(track_id)
        
        artists_ = []
        for artist in track['artists']:
            artists_.append(artist['name'])
        
        album = track['album']['name']
        album_id = track['album']['id']

        track_features = get_track_feature(track_id)
        
        # Create a dictionary representing the row of data
        row = {
    'Song_id': track_id,
    'Song': track_name,
    'Artist': ', '.join(artists_),
    'Album': album,
    'Album_id': album_id,
    'danceability': track_features['danceability'],
    'energy': track_features['energy'],
    'key': track_features['key'],
    'mode': track_features['mode'],
    'instrumentalness': track_features['instrumentalness'],
    'valence': track_features['valence'],
    'duration_ms': track_features['duration_ms'],
    'time_signature': track_features['time_signature'],
    'loudness': track_features['loudness'],
    'speechiness': track_features['speechiness'],
    'acousticness': track_features['acousticness'],
    'liveness': track_features['liveness'],
    'tempo': track_features['tempo'],
    'popularity': track['popularity'],
    'year_of_release': track['album']['release_date']
}
        
        # Append the row to the list
        rows.append(row)
    
    # Convert the list of rows into a DataFrame
    df_playlist = pd.DataFrame(rows)
    
    return df_playlist


In [12]:
def add_artist(playlist, df_artists):
    artists_id = []
    
    for item in playlist['tracks']['items']:
        for artist in item['track']['artists']:
            artists_id.append(artist['id'])
            
    # Check if df_artists is None or not a DataFrame
    if df_artists is None or not isinstance(df_artists, pd.DataFrame):
        df_artists = pd.DataFrame(columns=['Artist_id', 'name', 'followers', 'genres', 'popularity'])

    # Initialize a list to hold the rows of data
    rows = []

    for artist_id in artists_id:
        artist = get_unique_artist(artist_id)
        
        # Create a dictionary representing the row of data
        row = {'Artist_id': artist['id'], 'name': artist['name'], 'followers': artist['followers']['total'], 
               'genres': artist['genres'], 'popularity': artist['popularity']}
        
        # Append the row to the list
        rows.append(row)

    # Convert the list of rows into a DataFrame
    df_artists = pd.DataFrame(rows)
    
    return df_artists


In [13]:
playlist_id= '0dv7CtS9KKTx3f4TOkwI9i?si=a379b2e392e64048'
playlist= get_playlist_tracks (access_token, playlist_id)

In [14]:
df = pd.DataFrame(columns=['Song_id', 'Song', 'Artist', 'Album', 'Album_id', 'danceability', 'energy', 'key', 'mode', 'instrumentalness', 'valence', 'duration', 'time_signature', 'loudness', 'speechiness', 'acousticness', 'liveness', 'tempo', 'popularity', 'year'])
df

,Song_id,Song,Artist,Album,Album_id,danceability,energy,key,mode,instrumentalness,valence,duration,time_signature,loudness,speechiness,acousticness,liveness,tempo,popularity,year


In [15]:
df_tracks= add_playlist(playlist, df)

In [16]:
df_tracks.head()

,Song_id,Song,Artist,Album,Album_id,danceability,energy,key,mode,instrumentalness,valence,duration_ms,time_signature,loudness,speechiness,acousticness,liveness,tempo,popularity,year_of_release
0,1oacrRLAurXf6ylXFhF2YT,Chewing Gum,NCT DREAM,Chewing Gum,53wPhgvCnPI6QMvF7SABkX,0.774,0.800,3,0,0.0,0.931,197462,4,-3.426,0.0472,0.1130,0.0782,103.977,51,2016-08-27
1,24PhZrRiALtUXgdwXkEwGt,마지막 첫사랑 My First and Last,NCT DREAM,The First - The 1st Single Album,4wmCJoHoVwHdSBnLq5d4Sm,0.799,0.899,2,1,0.0,0.589,201980,4,-3.945,0.1310,0.0180,0.0801,108.051,52,2017-02-09
2,7lMMCxTBJU0ioAymYX8h03,最后的初恋 My First and Last - Chinese Version,NCT DREAM,The First - The 1st Single Album,4wmCJoHoVwHdSBnLq5d4Sm,0.801,0.885,9,0,0.0,0.571,201980,4,-4.108,0.1680,0.0254,0.0577,108.020,24,2017-02-09
3,2sOz5KStj3M1L1ErNo6MCC,덩크슛 Dunk Shot,NCT DREAM,The First - The 1st Single Album,4wmCJoHoVwHdSBnLq5d4Sm,0.644,0.888,2,1,0.0,0.651,232726,4,-4.097,0.0710,0.1240,0.2690,115.983,45,2017-02-09
4,2nChvM2Tyv1ib8AvfCJcOe,We Young,NCT DREAM,We Young - The 1st Mini Album,2cEYyauffphZuDONGi6z9b,0.712,0.857,0,1,0.0,0.338,224467,4,-4.194,0.0414,0.0579,0.0684,123.041,47,2017-08-17


In [17]:
df_tracks.to_csv("playlist_track_dream.csv", index=False)